In [13]:
import csv
import numpy as np
import pandas as pd
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pickle
import sys
sys.path.insert(0, r'/Users/xintianhan/Downloads/explanation/')
from explanations.explainer_new import Explainer


# get explanations

In [17]:
def export_explanations(explanations, labels, scores, features):
    with open('../files/explanations_avg.csv', 'w', newline='') as export_file:
        writer = csv.writer(export_file)
        writer.writerow(["Observation", "Label", "Prediction", "Explanation"])
        for i_e, e_list in enumerate(explanations):
            if len(e_list) > 0:
                for explanation in e_list:
                    row = [i_e, labels[i_e], scores[i_e]] + list(features[explanation])
                    writer.writerow(row)
            else:
                row = [i_e, labels[i_e], scores[i_e], "No Explanation"]
                writer.writerow(row)


def main():
    X_train, y_train, X_test, y_test = pickle.load( open( "../Data/LC_data.pickle", "rb" ) )
    feature_types = pickle.load(open("../Data/feature_types.pickle", "rb"))
    feature_types = np.array(feature_types)
    features = pickle.load(open("../Data/features.pickle", "rb"))
#     input_file = "files/readyToGo.csv"
    data_file = "files/cache/data.pkl"
#     labels_file = "files/cache/labels.pkl"
#     features_file = "files/cache/features.pkl"
    model_file = "../files/cache/model.pkl"
#     try:
#         data = pickle.load(open(data_file, "rb"))
#         labels = pickle.load(open(labels_file, "rb"))
#         features = pickle.load(open(features_file, "rb"))
#     except IOError:
#         data, labels, features = read_csv(input_file)
#         pickle.dump(data, open(data_file, "wb"))
#         pickle.dump(labels, open(labels_file, "wb"))
#         pickle.dump(features, open(features_file, "wb"))
    data = X_train
    labels = y_train
    features = np.array(features)
    try:
        model = pickle.load(open(model_file, "rb"))
    except IOError:
        print("Start fit")
        # model = LogisticRegression()
        model = LogisticRegression(penalty = 'l1', C = 4.641588833612778)
        model.fit(data, labels)
        print("Finish fitting")
        pickle.dump(model, open(model_file, "wb"))
    top_obs = 1000
    data = data[:top_obs, :]
    labels = labels[:top_obs]
    explainer = Explainer(model.predict_proba, 0.5134914215280924)
    max_ite = 200
    explanations = explainer.explain(data, feature_types, max_ite)
    scores = model.predict_proba(data)[:, 1]
    export_explanations(explanations, labels, scores, features)


main()